In [1]:
import os
import sys
import numpy as np
import pandas as pd
import datetime
import time
import itertools
import pickle

from sklearn import linear_model
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error
def mean_absolute_percentage_error(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

path_main_folder = '/home/antorosi/Documents/Prediction'
sys.path.append(path_main_folder)

from conso.load_shape_data import load_data_conso, get_uniformed_data_conso, change_granularity, get_x_y_prediction_conso, get_train_test_sets, normalized_dataset, select_variables

### Load and shape data 

In [10]:
# Load
path_data = os.path.join(path_main_folder, 'data')
dict_data_conso = load_data_conso(path_data)

# Uniformization
data_conso_df, dict_colnames_conso = get_uniformed_data_conso(dict_data_conso)

# Granularity from 15 min to 1H
data_conso_df = change_granularity(data_conso_df, granularity="1H")

# Get x and y from prediction
x_conso, y_conso, dict_colnames_conso = get_x_y_prediction_conso(data_conso_df, dict_colnames_conso, lags=[24,48])

### Cross-validation parameters 

In [11]:
# folder to store results
path_out = os.path.join(path_main_folder, 'out', 'benchmark_cmca_rf')
if not os.path.exists(path_out):
    os.mkdir(path_out)

In [12]:
# variables used for input
selected_variables = ['conso', 'calendar', 'meteo']
gen_name = 'cmca'

In [13]:
# Test periods for each K step of the cross-validation
cv_periods = {}
cv_periods['period_1'] = (datetime.datetime(year=2013, month=1, day=1), datetime.datetime(year=2013, month=12, day=31))
cv_periods['period_2'] = (datetime.datetime(year=2014, month=1, day=1), datetime.datetime(year=2014, month=12, day=31))
cv_periods['period_3'] = (datetime.datetime(year=2015, month=1, day=1), datetime.datetime(year=2015, month=12, day=31))
cv_periods['period_4'] = (datetime.datetime(year=2016, month=1, day=1), datetime.datetime(year=2016, month=12, day=31))
cv_periods['period_5'] = (datetime.datetime(year=2017, month=1, day=1), datetime.datetime(year=2017, month=12, day=31))


In [14]:
# Getting each datasets
dict_datasets = {}
for key, date_period in cv_periods.items():
    x_conso_selected_var = select_variables(x_conso, dict_colnames_conso, selected_variables)
    dataset, dict_ds = get_train_test_sets(x_conso_selected_var, y_conso, date_period[0], date_period[1])
    dataset = normalized_dataset(dataset, dict_colnames_conso)
    
    dict_datasets[key] = {'dataset': dataset, 'dict_ds': dict_ds}

### Dataset

In [15]:
date_period = [datetime.datetime(2013,1,1), datetime.datetime(2017,12,31)]

x_conso_selected_var = select_variables(x_conso, dict_colnames_conso, selected_variables)
dataset, dict_ds = get_train_test_sets(x_conso_selected_var, y_conso, date_period[0], date_period[1])
dataset = normalized_dataset(dataset, dict_colnames_conso)

dataset = dataset['test']
dict_ds=dict_ds['test']

x = dataset['x']
y = dataset['y']

In [16]:
train_indices = list()
test_indices = list()

for year in [2013,2014,2015,2016,2017]:
    mask = dict_ds.dt.year == year
    test_indices.append(np.where(mask)[0])
    train_indices.append(np.where(np.invert(mask))[0])

custom_cv = zip(train_indices, test_indices)

In [17]:
from sklearn.model_selection import RandomizedSearchCV

# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 50, stop = 400, num = 8)]
# Number of features to consider at every split
max_features = ['sqrt', 1/3]
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4, 6, 8, 10, 15]
# Method of selecting samples for training each tree
bootstrap = [True]

# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf}

In [18]:
rf = RandomForestRegressor()
# Random search of parameters, using 3 fold cross validation, 
# search across 100 different combinations, and use all available cores
rf_random = RandomizedSearchCV(scoring='neg_mean_squared_error', estimator = rf, param_distributions = random_grid, n_iter = 250, cv = custom_cv, verbose=2, random_state=44, n_jobs = -1)

# Fit the random search model
rf_random.fit(X=x, y=np.ravel(y))

Fitting 5 folds for each of 250 candidates, totalling 1250 fits
[CV] max_depth=110, min_samples_split=5, min_samples_leaf=15, max_features=sqrt, n_estimators=100 
[CV] max_depth=110, min_samples_split=5, min_samples_leaf=15, max_features=sqrt, n_estimators=100 
[CV] max_depth=110, min_samples_split=5, min_samples_leaf=15, max_features=sqrt, n_estimators=100 
[CV] max_depth=110, min_samples_split=5, min_samples_leaf=15, max_features=sqrt, n_estimators=100 
[CV] max_depth=110, min_samples_split=5, min_samples_leaf=15, max_features=sqrt, n_estimators=100 
[CV] max_depth=20, min_samples_split=10, min_samples_leaf=2, max_features=sqrt, n_estimators=50 
[CV] max_depth=20, min_samples_split=10, min_samples_leaf=2, max_features=sqrt, n_estimators=50 
[CV] max_depth=20, min_samples_split=10, min_samples_leaf=2, max_features=sqrt, n_estimators=50 
[CV]  max_depth=20, min_samples_split=10, min_samples_leaf=2, max_features=sqrt, n_estimators=50, total=  18.1s
[CV] max_depth=20, min_samples_split=1

[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:  3.5min


[CV]  max_depth=80, min_samples_split=2, min_samples_leaf=15, max_features=sqrt, n_estimators=400, total= 2.1min
[CV] max_depth=50, min_samples_split=10, min_samples_leaf=10, max_features=0.3333333333333333, n_estimators=300 
[CV]  max_depth=80, min_samples_split=2, min_samples_leaf=15, max_features=sqrt, n_estimators=400, total= 2.0min
[CV] max_depth=50, min_samples_split=10, min_samples_leaf=10, max_features=0.3333333333333333, n_estimators=300 
[CV]  max_depth=80, min_samples_split=2, min_samples_leaf=15, max_features=sqrt, n_estimators=400, total= 2.1min
[CV] max_depth=40, min_samples_split=5, min_samples_leaf=2, max_features=0.3333333333333333, n_estimators=350 
[CV]  max_depth=80, min_samples_split=2, min_samples_leaf=15, max_features=sqrt, n_estimators=400, total= 2.1min
[CV] max_depth=40, min_samples_split=5, min_samples_leaf=2, max_features=0.3333333333333333, n_estimators=350 
[CV]  max_depth=80, min_samples_split=2, min_samples_leaf=15, max_features=sqrt, n_estimators=400, t

[CV] max_depth=40, min_samples_split=10, min_samples_leaf=4, max_features=0.3333333333333333, n_estimators=150 
[CV]  max_depth=40, min_samples_split=10, min_samples_leaf=4, max_features=0.3333333333333333, n_estimators=150, total= 4.5min
[CV] max_depth=40, min_samples_split=10, min_samples_leaf=4, max_features=0.3333333333333333, n_estimators=150 
[CV]  max_depth=40, min_samples_split=10, min_samples_leaf=4, max_features=0.3333333333333333, n_estimators=150, total= 4.4min
[CV] max_depth=10, min_samples_split=2, min_samples_leaf=10, max_features=0.3333333333333333, n_estimators=350 
[CV]  max_depth=80, min_samples_split=2, min_samples_leaf=15, max_features=0.3333333333333333, n_estimators=400, total= 9.6min
[CV] max_depth=10, min_samples_split=2, min_samples_leaf=10, max_features=0.3333333333333333, n_estimators=350 
[CV]  max_depth=80, min_samples_split=2, min_samples_leaf=15, max_features=0.3333333333333333, n_estimators=400, total= 9.4min
[CV] max_depth=10, min_samples_split=2, min_

[CV] max_depth=10, min_samples_split=10, min_samples_leaf=6, max_features=0.3333333333333333, n_estimators=50 
[CV]  max_depth=100, min_samples_split=5, min_samples_leaf=8, max_features=0.3333333333333333, n_estimators=250, total= 6.4min
[CV] max_depth=50, min_samples_split=5, min_samples_leaf=8, max_features=sqrt, n_estimators=250 
[CV]  max_depth=100, min_samples_split=5, min_samples_leaf=8, max_features=0.3333333333333333, n_estimators=250, total= 6.5min
[CV] max_depth=50, min_samples_split=5, min_samples_leaf=8, max_features=sqrt, n_estimators=250 
[CV]  max_depth=100, min_samples_split=5, min_samples_leaf=8, max_features=0.3333333333333333, n_estimators=250, total= 6.5min
[CV] max_depth=50, min_samples_split=5, min_samples_leaf=8, max_features=sqrt, n_estimators=250 
[CV]  max_depth=10, min_samples_split=10, min_samples_leaf=6, max_features=0.3333333333333333, n_estimators=50, total=  56.6s
[CV] max_depth=50, min_samples_split=5, min_samples_leaf=8, max_features=sqrt, n_estimators

...

[CV]  max_depth=20, min_samples_split=10, min_samples_leaf=10, max_features=sqrt, n_estimators=100, total=  31.6s
[CV] max_depth=20, min_samples_split=10, min_samples_leaf=15, max_features=sqrt, n_estimators=250 
[CV]  max_depth=20, min_samples_split=10, min_samples_leaf=10, max_features=sqrt, n_estimators=100, total=  31.3s
[CV] max_depth=90, min_samples_split=5, min_samples_leaf=8, max_features=0.3333333333333333, n_estimators=400 
[CV]  max_depth=20, min_samples_split=10, min_samples_leaf=15, max_features=sqrt, n_estimators=250, total= 1.2min
[CV] max_depth=90, min_samples_split=5, min_samples_leaf=8, max_features=0.3333333333333333, n_estimators=400 
[CV]  max_depth=20, min_samples_split=10, min_samples_leaf=15, max_features=sqrt, n_estimators=250, total= 1.2min
[CV] max_depth=90, min_samples_split=5, min_samples_leaf=8, max_features=0.3333333333333333, n_estimators=400 
[CV]  max_depth=90, min_samples_split=10, min_samples_leaf=4, max_features=0.3333333333333333, n_estimators=250,

[CV]  max_depth=90, min_samples_split=10, min_samples_leaf=4, max_features=0.3333333333333333, n_estimators=200, total= 5.7min
[CV] max_depth=50, min_samples_split=5, min_samples_leaf=6, max_features=0.3333333333333333, n_estimators=350 
[CV]  max_depth=60, min_samples_split=2, min_samples_leaf=2, max_features=sqrt, n_estimators=250, total= 1.8min
[CV] max_depth=50, min_samples_split=5, min_samples_leaf=6, max_features=0.3333333333333333, n_estimators=350 
[CV]  max_depth=60, min_samples_split=2, min_samples_leaf=2, max_features=sqrt, n_estimators=250, total= 1.8min
[CV] max_depth=50, min_samples_split=5, min_samples_leaf=6, max_features=0.3333333333333333, n_estimators=350 
[CV]  max_depth=60, min_samples_split=2, min_samples_leaf=2, max_features=sqrt, n_estimators=250, total= 1.8min
[CV] max_depth=50, min_samples_split=5, min_samples_leaf=6, max_features=0.3333333333333333, n_estimators=350 
[CV]  max_depth=60, min_samples_split=2, min_samples_leaf=2, max_features=sqrt, n_estimators=

[CV] max_depth=90, min_samples_split=10, min_samples_leaf=1, max_features=sqrt, n_estimators=400 
[CV]  max_depth=80, min_samples_split=10, min_samples_leaf=6, max_features=0.3333333333333333, n_estimators=300, total= 8.0min
[CV] max_depth=90, min_samples_split=10, min_samples_leaf=1, max_features=sqrt, n_estimators=400 
[CV]  max_depth=60, min_samples_split=2, min_samples_leaf=2, max_features=sqrt, n_estimators=400, total= 2.9min
[CV] max_depth=90, min_samples_split=10, min_samples_leaf=1, max_features=sqrt, n_estimators=400 
[CV]  max_depth=60, min_samples_split=2, min_samples_leaf=2, max_features=sqrt, n_estimators=400, total= 2.8min
[CV] max_depth=90, min_samples_split=10, min_samples_leaf=1, max_features=sqrt, n_estimators=400 
[CV]  max_depth=60, min_samples_split=2, min_samples_leaf=2, max_features=sqrt, n_estimators=400, total= 2.8min
[CV] max_depth=30, min_samples_split=2, min_samples_leaf=10, max_features=0.3333333333333333, n_estimators=350 
[CV]  max_depth=60, min_samples_s

[Parallel(n_jobs=-1)]: Done 1250 out of 1250 | elapsed: 574.8min finished


RandomizedSearchCV(cv=<zip object at 0x7fbc90149848>, error_score='raise',
          estimator=RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
           oob_score=False, random_state=None, verbose=0, warm_start=False),
          fit_params=None, iid=True, n_iter=250, n_jobs=-1,
          param_distributions={'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110], 'min_samples_split': [2, 5, 10], 'min_samples_leaf': [1, 2, 4, 6, 8, 10, 15], 'n_estimators': [50, 100, 150, 200, 250, 300, 350, 400], 'max_features': ['sqrt', 0.3333333333333333]},
          pre_dispatch='2*n_jobs', random_state=44, refit=True,
          return_train_score='warn', scoring='neg_mean_squared_error',
          verbose=2)

In [19]:
with open(os.path.join(path_out,'rf_random.pickle'), 'wb') as f:
    pickle.dump(rf_random,f)

In [20]:
with open(os.path.join(path_out,'results.pickle'), 'wb') as f:
    pickle.dump(rf_random.cv_results_,f)

In [21]:
rf_random.best_estimator_

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=50,
           max_features=0.3333333333333333, max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=2, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=350, n_jobs=1,
           oob_score=False, random_state=None, verbose=0, warm_start=False)

In [22]:
rf_random.cv_results_

/local/home/antorosi/PycharmProjects/KERAS-TS-VENV/lib/python3.5/site-packages/sklearn/utils/deprecation.py:122: FutureWarning: You are accessing a training score ('mean_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
/local/home/antorosi/PycharmProjects/KERAS-TS-VENV/lib/python3.5/site-packages/sklearn/utils/deprecation.py:122: FutureWarning: You are accessing a training score ('split0_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
/local/home/antorosi/PycharmProjects/KERAS-TS-VENV/lib/python3.5/site-packages/sklearn/utils/deprecation.py:122: FutureWarning: You are accessing a training score ('split1_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_sco

{'mean_fit_time': array([ 28.28680711,  17.93715644,  82.87584863,  78.26734614,
         91.32864842, 122.93141813, 459.90646863, 696.09640703,
        422.15685792, 114.68304887,  67.52193785,  69.71517501,
        568.3252028 , 269.31253443, 417.433147  ,  80.93252769,
         89.48139324,  91.35542912, 388.61765847, 203.48246579,
         57.15275712,  85.4151341 ,  96.82242589, 537.58756628,
         18.16577644, 118.77424245, 455.08743305,  72.01061764,
        547.58183408, 401.31367054, 136.89607797,  63.59011726,
         16.65743413,  99.84665084, 116.15026751,  72.97852321,
         87.69750199, 874.43356843,  52.4013556 ,  88.32072306,
         17.98229799, 115.68555326, 280.98331251, 220.00359359,
         99.93625388,  50.95237393,  60.26300054,  76.17321091,
        539.25898848,  73.00129986,  24.82563729, 102.92887554,
        363.18867722,  11.05194592, 106.20143256,  44.27922769,
        136.37274008, 539.39469337, 135.25032787, 184.75430789,
         87.25552115,  

In [23]:
rf_random.grid_scores_

/local/home/antorosi/PycharmProjects/KERAS-TS-VENV/lib/python3.5/site-packages/sklearn/model_selection/_search.py:761: DeprecationWarning: The grid_scores_ attribute was deprecated in version 0.18 in favor of the more elaborate cv_results_ attribute. The grid_scores_ attribute will not be available from 0.20
  DeprecationWarning)


[mean: -9708438.91081, std: 559781.26918, params: {'max_features': 'sqrt', 'min_samples_leaf': 15, 'n_estimators': 100, 'max_depth': 110, 'min_samples_split': 5},
 mean: -8572617.96398, std: 613499.63001, params: {'max_features': 'sqrt', 'min_samples_leaf': 2, 'n_estimators': 50, 'max_depth': 20, 'min_samples_split': 10},
 mean: -8711092.13190, std: 567933.81059, params: {'max_features': 'sqrt', 'min_samples_leaf': 8, 'n_estimators': 250, 'max_depth': 90, 'min_samples_split': 10},
 mean: -8262165.84540, std: 472509.97819, params: {'max_features': 'sqrt', 'min_samples_leaf': 4, 'n_estimators': 200, 'max_depth': 40, 'min_samples_split': 5},
 mean: -8123972.70421, std: 509900.91821, params: {'max_features': 'sqrt', 'min_samples_leaf': 2, 'n_estimators': 200, 'max_depth': 40, 'min_samples_split': 5},
 mean: -9639020.17593, std: 622708.07210, params: {'max_features': 'sqrt', 'min_samples_leaf': 15, 'n_estimators': 400, 'max_depth': 80, 'min_samples_split': 2},
 mean: -2911362.71671, std: 31